In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.gmm import GMM
from datetime import datetime

In [2]:
data = pd.read_csv('bus1234.csv')
data['day'] = 1
data['day'] = pd.to_datetime(data[['year','month','day']])
data['miles'] =data['miles']/1e6

print(data.columns)

Index(['Unnamed: 0', 'id', 'group', 'year', 'month', 'replace', 'miles',
       'day'],
      dtype='object')


In [9]:
#from statsmodels.tsa.arima_model import ARMA

#create ID dummies
#different lines for each

#model = ARMA(data, order=(0, 1))
#model_fit = model.fit(disp=False)



print( ((data['miles']*data['replace']).mean() - data['miles'].mean()*data['replace'].mean())/
      data['miles'].var() )



0.11721886920407547


In [12]:
class test(GMM):
    
    def __init__(self,*args, shift=50, gap=1, nmoms=10, **kwds, ):
        # set appropriate counts for moment conditions and parameters
        kwds.setdefault('k_moms', nmoms)
        kwds.setdefault('k_params',2)
        self.shift = shift
        self.gap = gap
        super(test, self).__init__(*args, **kwds)
        
        
    def momcond(self, params):
        
        data = self.instrument.copy()
        data['shock'] =  data['replace']*data['shock'] - 1*(data['replace']==0)*data['shock']
        data['miles'] = data['miles']
        data['util'] = params[0]*data['miles']*(1-data['replace']) + params[1]*data['replace']
        data['util_shock'] = data['shock'] + data['util']
        u_avg = data['util'].mean()
        
        moms = []
        for t in range(1,self.nmoms+1):
            #lag but loop around
            u_lag = data.groupby('id')['util'].shift(t*self.gap+self.shift)
            moms.append(data['util_shock']*u_lag - u_avg**2)
        moms = pd.DataFrame(np.array(moms).transpose())
        moms = moms.dropna()
        return moms
    
data['shock'] = sm.OLS(data['replace'], sm.add_constant( data['miles'] ) ).fit().resid
#data['shock'] = sm.Logit(data['replace'], sm.add_constant( data['miles']/1e6 ) ).fit().resid_response

data = data.sort_values(['id','day'])  
model = test(data['replace'],data['miles'],data[['id','miles','replace','day','shock']], shift=50)
result = model.fit(np.array([-.5,-10]),optim_method='nm')
result.summary()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 52
         Function evaluations: 103
Optimization terminated successfully.
         Current function value: 0.000253
         Iterations: 22
         Function evaluations: 43
Optimization terminated successfully.
         Current function value: 0.000253
         Iterations: 20
         Function evaluations: 41
Optimization terminated successfully.
         Current function value: 0.000253
         Iterations: 20
         Function evaluations: 41


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 test Results                                 
==============================================================================
Dep. Variable:                replace   Hansen J:                       0.6545
Model:                           test   Prob (Hansen J):                  1.00
Method:                           GMM                                         
Date:                Tue, 05 Nov 2019                                         
Time:                        11:39:23                                         
No. Observations:                8260                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0            0.0352      0.003     11.071      0.000       0.029       0.041
p 1           -0.9523      0.050    -19.179      0.000      -1.050      -0.855
==============================================================================
"""

In [7]:
N = 1000
ids = np.ones(N)
xs = np.zeros(N)
ys = np.zeros(N)
RC = -10
theta = -1



for i in range(1,N):

    ys[i] = (RC >= theta*xs[i-1] + np.random.uniform(-2,2) )
    
    if ys[i] == 0:
        xs[i] = xs[i-1] + np.random.uniform(0,1)
    if ys[i]==1:
        xs[i] = np.random.uniform(0,2)


        
data2 = pd.DataFrame( np.array([ids,xs,ys]).transpose(), columns=['id','miles','replace'])    
data2['shock'] = sm.OLS(data2['replace'], sm.add_constant( data2['miles'] ) ).fit().resid
#data['shock'] = sm.Logit(data['replace'], sm.add_constant( data['miles']/1e6 ) ).fit().resid_response
model = test(data2['replace'],data2['miles'],data2[['id','miles','replace','shock']] ,
             shift=100,gap=11,nmoms=3)
result = model.fit(np.array([-10,-2]),optim_method='nm')
result.summary()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 65
         Function evaluations: 126
Optimization terminated successfully.
         Current function value: 0.000550
         Iterations: 11
         Function evaluations: 22
Optimization terminated successfully.
         Current function value: 0.000269
         Iterations: 8
         Function evaluations: 16
Optimization terminated successfully.
         Current function value: 0.000020
         Iterations: 16
         Function evaluations: 29
Optimization terminated successfully.
         Current function value: 0.000130
         Iterations: 9
         Function evaluations: 18


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 test Results                                 
==============================================================================
Dep. Variable:                replace   Hansen J:                       0.1125
Model:                           test   Prob (Hansen J):                 0.737
Method:                           GMM                                         
Date:                Tue, 05 Nov 2019                                         
Time:                        09:03:25                                         
No. Observations:                1000                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0         2.154e-09   1.75e-09      1.233      0.218   -1.27e-09    5.58e-09
p 1        -3.292e-08    2.8e-08     -1.176      0.239   -8.78e-08    2.19e-08
==============================================================================
"""